# Estudo de Caso: Eleições no Brasil

O Repositório de dados eleitorais é uma compilação de informações brutas das eleições, voltada para pesquisadores, imprensa e pessoas interessadas em analisar os dados de eleitorado, candidaturas, resultados e prestação de contas.

Neste Estudo de Caso, iremos analisar as informações acerca do perfil dos candidatos nas eleições de 2014 e 2018 no âmbito estadual e federal, declarações de bens e dados sobre os partidos, as coligações e as vagas por cargo e por unidade eleitoral.

Os dados foram retirados do [Repositório de Dados Eleitorais do TSE](http://www.tse.jus.br/eleicoes/estatisticas/repositorio-de-dados-eleitorais-1).

Abaixo as bases de dados das eleições de 2014 e 2018 são baixadas do repositório de dados do TSE:

In [3]:
%sh
if [ ! -d /databricks/driver/eleicoes ]; then
  mkdir /databricks/driver/eleicoes
fi

# Baixa o arquivo que contém as informações dos candidatos de 2014
if [ ! -f /databricks/driver/consulta_cand_2014.zip ]; then 
  wget http://agencia.tse.jus.br/estatistica/sead/odsele/consulta_cand/consulta_cand_2014.zip
  unzip /databricks/driver/consulta_cand_2014.zip -d eleicoes/consulta_cand_2014
fi

# Baixa o arquivo que contém as informações dos bens dos candidatos de 2014
if [ ! -f /databricks/driver/bem_candidato_2014.zip ]; then 
  wget http://agencia.tse.jus.br/estatistica/sead/odsele/bem_candidato/bem_candidato_2014.zip
  unzip /databricks/driver/bem_candidato_2014.zip -d eleicoes/bem_candidato_2014
fi

# Baixa o arquivo que contém as informações dos candidatos de 2018
if [ ! -f /databricks/driver/consulta_cand_2018.zip ]; then 
  wget http://agencia.tse.jus.br/estatistica/sead/odsele/consulta_cand/consulta_cand_2018.zip
  unzip /databricks/driver/consulta_cand_2018.zip -d eleicoes/consulta_cand_2018
fi

# Baixa o arquivo que contém as informações dos bens dos candidatos de 2018
if [ ! -f /databricks/driver/bem_candidato_2018.zip ]; then 
  wget http://agencia.tse.jus.br/estatistica/sead/odsele/bem_candidato/bem_candidato_2018.zip
  unzip /databricks/driver/bem_candidato_2018.zip -d eleicoes/bem_candidato_2018
fi

In [4]:
%sh ls /databricks/driver/eleicoes/bem_candidato_2014

In [5]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [6]:
bem_candidato_schema = StructType([StructField("DT_GERACAO", DateType(), True),
                    StructField("HH_GERACAO", StringType(), True),
                    StructField("ANO_ELEICAO", IntegerType(), True),
                    StructField("CD_TIPO_ELEICAO", IntegerType(), True),
                    StructField("NM_TIPO_ELEICAO", StringType(), True),
                    StructField("CD_ELEICAO", IntegerType(), True),
                    StructField("DS_ELEICAO", StringType(), True),
                    StructField("DT_ELEICAO", DateType(), True),
                    StructField("SG_UF", StringType(), True),
                    StructField("SG_UE", StringType(), True),
                    StructField("NM_UE", StringType(), True),
                    StructField("SQ_CANDIDATO", LongType(), True),
                    StructField("NR_ORDEM_CANDIDATO", IntegerType(), True),
                    StructField("CD_TIPO_BEM_CANDIDATO", IntegerType(), True),
                    StructField("DS_TIPO_BEM_CANDIDATO", StringType(), True),
                    StructField("DS_BEM_CANDIDATO", StringType(), True),
                    StructField("VR_BEM_CANDIDATO", StringType(), True),
                    StructField("DT_ULTIMA_ATUALIZACAO", DateType(), True),
                    StructField("HH_ULTIMA_ATUALIZACAO", StringType(), True)])

In [7]:
bem_candidato_2014_df = spark.read.csv('file:///databricks/driver/eleicoes/bem_candidato_2014/bem_candidato_2014_GO.csv', header=True, encoding='Latin1', sep=';', schema=bem_candidato_schema, dateFormat='dd/MM/yyyy')
bem_candidato_2018_df = spark.read.csv('file:///databricks/driver/eleicoes/bem_candidato_2018/bem_candidato_2018_GO.csv', header=True, encoding='Latin1', sep=';', schema=bem_candidato_schema, dateFormat='dd/MM/yyyy')
consulta_cand_2014_df = spark.read.csv('file:///databricks/driver/eleicoes/consulta_cand_2014/consulta_cand_2014_GO.csv', header=True, encoding='Latin1', sep=';', inferSchema=True, timestampFormat='dd/MM/yyyy')
consulta_cand_2018_df = spark.read.csv('file:///databricks/driver/eleicoes/consulta_cand_2018/consulta_cand_2018_GO.csv', header=True, encoding='Latin1', sep=';', inferSchema=True, timestampFormat='dd/MM/yyyy')


Antes de tudo, devemos substituir os valores igual a #NULO# por ''(vazio) nas colunas 'NM_SOCIAL_CANDIDATO' e 'DS_SIT_TOT_TURNO' dos DataFrames consulta_cand_2014_df e consulta_cand_2018_df. 
Além disso, temos que substituir os valores igual a #NULO# por ''(vazio) na coluna 'DS_BEM_CANDIDATO' dos DataFrames bem_candidato_2014_df e bem_candidato_2018_df.

In [9]:
# candidatos 2014
consulta_cand_2014_df = consulta_cand_2014_df.withColumn('DS_SIT_TOT_TURNO', regexp_replace('DS_SIT_TOT_TURNO', '#NULO#', ''))
consulta_cand_2014_df = consulta_cand_2014_df.withColumn('NM_SOCIAL_CANDIDATO', regexp_replace('NM_SOCIAL_CANDIDATO', '#NULO#', ''))
# candidatos 2018
consulta_cand_2018_df = consulta_cand_2018_df.withColumn('DS_SIT_TOT_TURNO', regexp_replace('DS_SIT_TOT_TURNO', '#NULO#', ''))
consulta_cand_2018_df = consulta_cand_2018_df.withColumn('NM_SOCIAL_CANDIDATO', regexp_replace('NM_SOCIAL_CANDIDATO', '#NULO#', ''))
# bens dos candidatos de 2014
bem_candidato_2014_df = bem_candidato_2014_df.withColumn('DS_BEM_CANDIDATO', regexp_replace('DS_BEM_CANDIDATO', '#NULO#', ''))
# bens dos candidatos de 2018
bem_candidato_2018_df = bem_candidato_2018_df.withColumn('DS_BEM_CANDIDATO', regexp_replace('DS_BEM_CANDIDATO', '#NULO#', ''))


In [10]:
display(consulta_cand_2014_df)

In [11]:
# Quais candidatos foram eleitos em 2014 no Estado?
eleitos_2014 = consulta_cand_2014_df.filter(col('CD_SIT_TOT_TURNO').isin('1','2','3')).select('NM_CANDIDATO', 'NR_CPF_CANDIDATO', 'SQ_CANDIDATO', 'DS_CARGO')
display(eleitos_2014)

In [12]:
# Quais candidatos foram eleitos em 2018 no Estado?
eleitos_2018 = consulta_cand_2018_df.filter(col('CD_SIT_TOT_TURNO').isin('1','2','3')).select('NM_CANDIDATO', 'NR_CPF_CANDIDATO', 'SQ_CANDIDATO', 'DS_CARGO', 'ST_REELEICAO')
display(eleitos_2018)

In [13]:
# Qual o número de eleitos por partido em 2014 ordenando a saída pelo número de eleitos?
display(consulta_cand_2014_df.filter(col('CD_SIT_TOT_TURNO').isin('1','2','3')).groupBy(['NM_PARTIDO']).count().sort(desc('count')))

In [14]:
# Qual o número de eleitos por partido em 2018 ordenando a saída pelo número de eleitos?
display(consulta_cand_2018_df.filter(col('CD_SIT_TOT_TURNO').isin('1','2','3')).groupBy(['NM_PARTIDO']).count().sort(desc('count')))

In [15]:
# Quais candidatos eleitos em 2014 foram eleitos também em algum cargo eletivo em 2018 no Estado? Por quê o atributo 'ST_REELEICAO' nem sempre reflete a realidade?
display(eleitos_2014.join(eleitos_2018, 'NR_CPF_CANDIDATO'))

In [16]:
# Na eleição de 2014, quais eleitos estão no Top 50 com maior valor em Reais de bens declarados?
bem_candidato_2014_df = bem_candidato_2014_df.withColumn('VR_BEM_CANDIDATO', regexp_replace('VR_BEM_CANDIDATO', ',', '.').cast('double'))
t1 = bem_candidato_2014_df.sort(desc('VR_BEM_CANDIDATO')).limit(50)
t2 = consulta_cand_2014_df.filter(col('CD_SIT_TOT_TURNO').isin('1','2','3'))
display(t1.join(t2, 'SQ_CANDIDATO').dropDuplicates(['NM_CANDIDATO']))

In [17]:
# Na eleição de 2018, quais eleitos estão no Top 50 com maior valor em Reais de bens declarados?
bem_candidato_2018_df = bem_candidato_2018_df.withColumn('VR_BEM_CANDIDATO', regexp_replace('VR_BEM_CANDIDATO', ',', '.').cast('double'))
t1 = bem_candidato_2018_df.sort(desc('VR_BEM_CANDIDATO')).limit(50)
t2 = consulta_cand_2018_df.filter(col('CD_SIT_TOT_TURNO').isin('1','2','3'))
display(t1.join(t2, 'SQ_CANDIDATO').dropDuplicates(['NM_CANDIDATO']))

In [18]:
#escrevendo no formato Parquet
consulta_cand_2014_df.write.partitionBy('SG_PARTIDO').parquet('/tmp/cand_2014', mode='overwrite')
consulta_cand_2018_df.write.partitionBy('SG_PARTIDO').parquet('/tmp/cand_2018', mode='overwrite')


In [19]:
%sh
ls /dbfs/tmp/cand_2014

In [20]:
%sh
ls /dbfs/tmp/cand_2018